<a href="https://colab.research.google.com/github/Kiduhu/Cavin/blob/main/Copy_of_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import libraries and dependencies**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("algord/fake-news")

print("Path to dataset files:", path)

100%|██████████| 1.68M/1.68M [00:00<00:00, 95.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/algord/fake-news/versions/1


In [ ]:
path

'/root/.cache/kagglehub/datasets/algord/fake-news/versions/1'

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PREPROCESSING

In [ ]:
# load data into dataframe
news_dataset=pd.read_csv('/content/fake_or_real_news.csv')

In [ ]:
news_dataset.shape

(6335, 4)

In [ ]:
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# changing the categorical dataset into numerical
news_dataset['label'].replace(['FAKE','REAL'],[0,1],inplace=True)

<ipython-input-20-6e2f4cfc2c70>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  news_dataset['label'].replace(['FAKE','REAL'],[0,1],inplace=True)
<ipython-input-20-6e2f4cfc2c70>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  news_dataset['label'].replace(['FAKE','REAL'],[0,1],inplace=True)


In [ ]:
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [ ]:
news_dataset.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

In [ ]:
# counting number of missing values in dataset
news_dataset.isnull().sum()

,0
ID,0
title,0
text,0
label,0


In [ ]:
# separating the data
X=news_dataset.drop(columns='label')
Y=news_dataset['label']
print(X)
print(Y)

         ID  ...                                               text
0      8476  ...  Daniel Greenfield, a Shillman Journalism Fello...
1     10294  ...  Google Pinterest Digg Linkedin Reddit Stumbleu...
2      3608  ...  U.S. Secretary of State John F. Kerry said Mon...
3     10142  ...  — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       875  ...  It's primary day in New York and front-runners...
...     ...  ...                                                ...
6330   4490  ...  The State Department told the Republican Natio...
6331   8062  ...  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332   8622  ...   Anti-Trump Protesters Are Tools of the Oligar...
6333   4021  ...  ADDIS ABABA, Ethiopia —President Obama convene...
6334   4330  ...  Jeb Bush Is Suddenly Attacking Trump. Here's W...

[6335 rows x 3 columns]
0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int64



Stemming procedure

stemming is the process of reducing a word to its root word

example....
actor, acting, actress ---> act


In [ ]:
port_stem=PorterStemmer()

In [ ]:
# create a function
def stemming(title):
  stemmed_title= re.sub('[^a-zA-Z]', ' ',title)
  stemmed_title=stemmed_title.lower()
  stemmed_title=stemmed_title.split()
  stemmed_title=[port_stem.stem(word) for word in stemmed_title if not word in stopwords.words('english')]
  stemmed_title= ' '.join(stemmed_title)
  return stemmed_title

In [ ]:
news_dataset['title']=news_dataset['title'].apply(stemming)

In [ ]:
print(news_dataset['title'])

0                                      smell hillari fear
1       watch exact moment paul ryan commit polit suic...
2                           kerri go pari gestur sympathi
3          berni support twitter erupt anger dnc tri warn
4                           battl new york primari matter
                              ...                        
6330       state depart say find email clinton specialist
6331                        p pb stand plutocrat pentagon
6332             anti trump protest tool oligarchi inform
6333    ethiopia obama seek progress peac secur east a...
6334                jeb bush suddenli attack trump matter
Name: title, Length: 6335, dtype: object


In [ ]:
#separating the data and label
X=news_dataset['title'].values
Y=news_dataset['label'].values

In [ ]:
print(X)
print(Y)

['smell hillari fear'
 'watch exact moment paul ryan commit polit suicid trump ralli video'
 'kerri go pari gestur sympathi' ...
 'anti trump protest tool oligarchi inform'
 'ethiopia obama seek progress peac secur east africa'
 'jeb bush suddenli attack trump matter']
[0 0 1 ... 0 1 1]


In [ ]:
# convert text to meaningful values using TF vectorizor
vectorizor=TfidfVectorizer()
vectorizor.fit(X)

X=vectorizor.transform(X)

In [ ]:
print(X)

  (0, 2230)	0.5606827254461214
  (0, 2817)	0.31703295849951846
  (0, 5631)	0.7649346276717717
  (1, 1212)	0.3538286538491304
  (1, 2101)	0.44595444447238664
  (1, 3939)	0.3279355637571237
  (1, 4462)	0.26757273402295384
  (1, 4630)	0.24403179029804972
  (1, 4905)	0.29382045650093463
  (1, 5263)	0.29163264316148546
  (1, 5933)	0.3501812302002013
  (1, 6315)	0.13561903129801023
  (1, 6571)	0.21736822482437437
  (1, 6671)	0.27855534550881683
  (2, 2536)	0.5345737509664292
  (2, 2576)	0.3357697230253524
  (2, 3335)	0.41740393299177314
  (2, 4428)	0.37615749278439464
  (2, 6015)	0.5345737509664292
  (3, 215)	0.41916879208304547
  (3, 561)	0.2969312816565521
  (3, 1764)	0.3507454498930073
  (3, 2048)	0.4448394256850793
  (3, 5956)	0.27803614912134916
  (3, 6288)	0.3148734995784654
  :	:
  (6330, 5809)	0.3024865273895357
  (6331, 4474)	0.5790025183139886
  (6331, 4493)	0.45939150101458504
  (6331, 4611)	0.5530926787707366
  (6331, 5796)	0.38445288563338503
  (6332, 240)	0.3633999482583336
  (

Splitting data into training and testing dataset

In [ ]:
X_train,Xtest,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
#train our model
model=LogisticRegression

In [ ]:
model.fit(X_train, Y_train)

AttributeError: 'csr_matrix' object has no attribute '_validate_params'